In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

from sklearn.model_selection import KFold, train_test_split

In [8]:
kfold_data_dir = './data/k-fold/'

In [9]:
file_list = os.listdir(kfold_data_dir)

In [11]:
print(len(file_list))

20


In [51]:
file_train_start = 'train'
file_test_start = 'test'

num_k = int(len(file_list) / 2)
print(f'Number of K is {num_k}')

predict_class_right = ['PostOp-SNR',  'PostOp-CNR',  'PostOp-BWR', 'PostOp-BHR',
                       'PostOp-IMFR',  'PostOp-AreRW', 'PostOp-AreRH']

predict_class_left = [ 'PostOp-SNL',  'PostOp-CNL', 'PostOp-BWR',
                      'PostOp-BHL',  'PostOp-IMFL','PostOp-AreLW', 'PostOp-AreLH']

predict_post_id = ['PostOp-ID']

feature_left = ['PostOp-SNL', 'PostOp-CNL', 'PostOp-BWL', 'PostOp-BHL', 'PostOp-IMFL', 'PostOp-AreLW',
                'PostOp-AreLH', 'PostOp-ID', 'ImplantLeft']

feature_right = ['PostOp-SNL', 'PostOp-CNL', 'PostOp-BWL', 'PostOp-BHL', 'PostOp-IMFL', 'PostOp-AreLW',
                'PostOp-AreLH', 'PostOp-ID', 'ImplantRight']

print(f'Number of model is {(len(predict_class_right) + len(predict_post_id) + len(predict_class_left))}')

Number of K is 10
Number of model is 15


In [64]:
df.head()

,OperationDate,Implant size,PreOp-SNR,PreOp-SNL,PreOp-CNR,PreOp-CNL,PreOp-BWR,PreOp-BWL,PreOp-BHR,PreOp-BHL,...,PostOp-BHL,PostOp-IMFR,PostOp-IMFL,PostOp-AreRW,PostOp-AreRH,PostOp-AreLW,PostOp-AreLH,PostOp-ID,ImplantLeft,ImplantRight
0,25-Nov-16,275,15.0,15.0,13.5,13.5,12.4,12.4,9.0,9.0,...,12.7,8.0,8.0,2.8,2.8,2.8,2.8,0.5,275,275
1,18-Nov-16,275,18.5,18.5,17.5,17.5,12.6,12.6,9.0,9.0,...,13.0,8.0,8.0,3.0,2.5,3.0,2.5,1.5,275,275
2,26-Aug-16,275,16.0,16.0,14.0,13.5,12.0,12.0,10.2,9.8,...,12.0,7.5,9.0,3.0,2.5,3.0,2.5,1.5,275,275
3,24-Sep-16,275,16.0,16.0,15.0,14.5,13.0,13.0,9.0,9.0,...,13.5,6.5,6.5,3.5,3.5,3.5,2.5,1.0,275,275
4,28-Oct-16,300,17.5,17.5,16.5,16.5,12.0,12.0,9.8,9.8,...,13.3,7.5,8.5,3.0,3.0,3.0,3.0,0.5,300,300


In [76]:
def get_x_and_y(df: pd.DataFrame, x_head=[], y_head=[]):
    x, y = [], []
    
    for index, row in df.iterrows():
        x_temp = []
        for x_h in x_head:
            x_temp.append(row[x_h])
        x.append(x_temp)
        y.append(row[y_head[0]])
    
    return x, y

In [84]:
result = {}

for k in range(num_k):
    train_df = pd.read_csv(os.path.join(kfold_data_dir, f'{file_train_start}_{k}.csv',))
    test_df = pd.read_csv(os.path.join(kfold_data_dir, f'{file_test_start}_{k}.csv'))
    
    # Create right side model
    
    x_train, y_train = get_x_and_y(train_df, feature_right, ['PostOp-SNL'])
    x_test, y_test = get_x_and_y(test_df, feature_right, ['PostOp-SNL'])
    print(len(x_test))

15
15
15
15
14
14
14
14
14
14
